In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import json
import numpy 
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline  
import pyarrow
import sys

from datetime import date
from dotenv import load_dotenv
from sqlalchemy import create_engine
from os import path
from typing import List,Dict, Tuple
from collections import defaultdict
pd.set_option("display.max_columns", None)

load_dotenv(verbose=True)
BIGQUERY_CREDENTIALS_PATH = os.environ.get('BIGQUERY_CREDENTIALS_PATH')
engine = create_engine('bigquery://bespoke-financial/ProdMetrcData', credentials_path=os.path.expanduser(BIGQUERY_CREDENTIALS_PATH))

sys.path.append(path.realpath(path.join(os.getcwd(), "../core")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../scripts")))
sys.path.append(path.realpath(path.join(os.getcwd(), "../../src")))

from underwriting import inventory_freshness_analyzer as ifa
from underwriting import inventory_freshness_analysis_query as ifa_query
import create_queries
import prepare_data

from bespoke.inventory.analysis.shared import download_util, inventory_types
from bespoke.inventory.analysis import active_inventory_util as util
from bespoke.inventory.analysis import inventory_valuations_util as valuations_util

%load_ext autoreload
%autoreload 2

In [3]:
TRANSFER_PACKAGES_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_START_DATE = '2020-01-01'
SALES_TRANSACTIONS_END_DATE = '2023-01-01'
today = date.today()

In [4]:
str(today)


'2022-05-20'

In [5]:
query_join = ifa_query.create_inventory_days_since_package_metric_query(
    ['EL', '99HT'], ['C9-0000370-LIC', 'C10-0000279-LIC'], True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE)
pd.read_sql_query(query_join, engine).head()
#print(query_join)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0188437424,active,Consumer,2022-05-18 19:04:43+00:00,2022-05-01 00:00:00+00:00,2,56.0,active,24704757,1A4060300022B79000319182,Sour Zkittlez Live Distillate 1g 510 Vape Cart...,Vape Cartridge (weight - each),Each,1.0,40.0,40.0,24704757,1A4060300022B79000319182,active,2022-04-28,2022-05-19 11:18:52+00:00,Product,Sour Zkittlez Live Distillate 1g 510 Vape Cart...,Vape Cartridge (weight - each),7.0,Each,4073698,Concentrate,,10322SZLRD,"Docs 7/22/21 GH10, Ice Cream Cake 7/22/21, Ice...",False,False,False,None,None,20,21
1,C9-0000370-LIC,0185588153,active,Consumer,2022-05-06 12:30:16+00:00,2022-05-01 00:00:00+00:00,3,100.0,active,24704567,1A4060300022B79000319217,"Snowcap 1g CDT Dist Vape (H), Hashtag Distribu...",Vape Cartridge (weight - each),Each,1.0,40.0,40.0,24704567,1A4060300022B79000319217,active,2022-04-28,2022-05-19 11:18:56+00:00,Product,"Snowcap 1g CDT Dist Vape (H), Hashtag Distribu...",Vape Cartridge (weight - each),17.0,Each,3769370,Concentrate,,32222CDTSNOCAP,"1-21, 10-15 peanut butter souffle, 10-6 cherry...",False,False,False,None,None,8,21
2,C9-0000370-LIC,0185588153,active,Consumer,2022-05-06 12:30:16+00:00,2022-05-01 00:00:00+00:00,3,100.0,active,24704626,1A4060300022B79000320195,"BBrand Melon Gum 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704626,1A4060300022B79000320195,active,2022-04-28,2022-05-17 11:18:41+00:00,Product,"BBrand Melon Gum 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),19.0,Each,3389928,Concentrate,,20222BDTMG,"""33"" 12-12-19, #19, #20, #21, #22, #24, #25, #...",False,False,False,None,None,8,19
3,C9-0000370-LIC,0188181528,active,Consumer,2022-05-17 11:37:23+00:00,2022-05-01 00:00:00+00:00,3,110.0,active,24704619,1A4060300022B79000320202,"BBrand Blueberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704619,1A4060300022B79000320202,active,2022-04-28,2022-05-18 11:18:44+00:00,Product,"BBrand Blueberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),15.0,Each,3389918,Concentrate,,31822BDTBLU,"1-21, 10-15 peanut butter souffle, 10-6 cherry...",False,False,False,None,None,19,20
4,C9-0000370-LIC,0188181528,active,Consumer,2022-05-17 11:37:23+00:00,2022-05-01 00:00:00+00:00,3,110.0,active,24704567,1A4060300022B79000319217,"Snowcap 1g CDT Dist Vape (H), Hashtag Distribu...",Vape Cartridge (weight - each),Each,1.0,40.0,40.0,24704567,1A4060300022B79000319217,active,2022-04-28,2022-05-19 11:18:56+00:00,Product,"Snowcap 1g CDT Dist Vape (H), Hashtag Distribu...",Vape Cartridge (weight - each),17.0,Each,3769370,Concentrate,,32222CDTSNOCAP,"1-21, 10-15 peanut butter souffle, 10-6 cherry...",False,False,False,None,None,19,21


In [6]:
query_metric = ifa_query.create_company_freshness_metric_query(['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE,SALES_TRANSACTIONS_END_DATE, 'tx_product_category_name')
print(query_metric)


        SELECT 
            tx_product_category_name,
            AVG(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS avg_days_since_sale,
            MAX(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS max_days_since_sale,
            MIN(CASE WHEN sales_days_diff > 0 THEN sales_days_diff ELSE 0 END) AS min_days_since_sale,
            COUNT(*) AS number_of_sales,
            AVG(price_per_unit) AS avg_sale_price
        FROM 
            (
        SELECT 
            sales.license_number,
            sales.receipt_number,
            sales.rt_type,
            sales.sales_customer_type,
            sales.sales_datetime,
            sales.sales_month,
            sales.total_packages,
            sales.rt_total_price,
            sales.tx_type,
            sales.tx_package_id,
            sales.tx_package_label,
            sales.tx_product_name,
            sales.tx_product_category_name,
            sales.tx_unit_of_measure,
            sales.tx_

In [7]:
pd.read_sql_query(query_metric, engine)

,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Tincture (volume - each),404.461538,512,76,13,46.006154
1,Edible (weight - each),107.823301,605,2,515,17.680466
2,Pre-Roll Leaf,66.674419,255,1,43,31.244186
3,Vape Cartridge (volume - each),36.927152,323,2,151,40.224636
4,Flower (packaged - each),23.354839,43,2,31,24.758065
5,Pre-Roll Flower,109.981818,287,7,275,32.374545
6,Vape Cartridge (weight - each),37.230530,183,1,321,39.221059
7,Capsule (weight - each),74.020408,281,2,49,21.212245
8,Flower (packaged eighth - each),72.783333,280,1,180,36.343444
9,Other Concentrate (weight - each),187.785714,434,4,28,34.505714


In [8]:
query_metric = ifa_query.create_company_freshness_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, 'tx_product_category_name')
pd.read_sql_query(query_metric, engine)

,tx_product_category_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,Flower (packaged quarter - each),40.145455,141,4,165,84.500000
1,Flower (packaged half ounce - each),41.500000,133,1,546,142.943681
2,Tincture (volume - each),113.728625,512,1,538,61.435325
3,Pre-Roll Leaf,40.682292,255,1,576,19.206641
4,Tincture (weight - each),11.000000,13,7,8,75.000000
5,Edible (volume - each),70.460432,268,0,417,15.031199
6,Seeds (each),96.000000,154,33,4,121.875000
7,Other Concentrate (volume - each),33.793103,55,4,29,29.896552
8,Flower (packaged - each),34.918322,133,1,453,54.095475
9,Topical (weight - each),89.150000,401,0,160,32.549375


In [9]:
query_metric = ifa_query.create_company_freshness_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, 'tx_product_name')
pd.read_sql_query(query_metric, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,avg_sale_price
0,ABX Maui Wowie Gummies (100mg),40.414530,98,4,234,14.961538
1,KE - THC Indica Strawberry 100mg - 43g,43.083333,81,9,12,20.000000
2,1 G Mango - Live Resin Sauce,58.714286,92,10,7,40.071429
3,Source Flower SFV OG Eighth 3.5g,17.692308,33,12,26,70.961538
4,FloraCal Gelato Mints Smalls Prepackaged 1/8th,63.160000,139,7,25,40.000000
...,...,...,...,...,...,...
751,(AG/FF) - (Cherry Slurry) - 1g Live Resin Vape...,117.666667,165,31,6,43.333333
752,STIIIZY - Blueberry Blast Curated live Resin -...,109.000000,139,82,6,40.000000
753,"Tutti 1g Sauce - Hybrid - Bubba Kush, Biscotti...",93.562500,230,16,16,30.000000
754,Source Flower Kosher Kush Half Ounces,30.000000,45,22,3,251.666667


In [10]:
query_valuation = ifa_query.create_company_inventory_valudation_by_last_sale_price(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE)
pd.read_sql_query(query_valuation, engine)

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank
0,11497141,25.60,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1
...,...,...,...,...,...,...,...,...
287,8164573,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
288,8164574,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
289,8486461,23.04,4.0,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),1.405479,92.16,1
290,9166114,64.00,1.0,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.358904,64.00,1


In [11]:
query_valuation = ifa_query.create_company_inventory_valudation_by_last_sale_price(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE,SALES_TRANSACTIONS_END_DATE, False)
pd.read_sql_query(query_valuation, engine)

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank
0,11497141,25.60,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1
...,...,...,...,...,...,...,...,...
224,7903512,12.80,4.0,Kushy Punch Recover Gummy,Edible (weight - each),1.504110,51.20,1
225,7903524,10.24,4.0,Kushy Punch Sativa Gummy,Edible (weight - each),1.660274,40.96,1
226,8486461,23.04,4.0,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),1.405479,92.16,1
227,9166114,64.00,1.0,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.358904,64.00,1


In [12]:
query_top_product_by_speed = ifa_query.best_selling_products_by_liquidity(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'], SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, 'tx_product_name',
    'avg_days_since_sale', 'ASC', '10')
pd.read_sql_query(query_top_product_by_speed, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,HIMALAYA Live Sauce Carts - Kimbo Kush (I) 500...,0.909091,1,0,11,12.0,63.637727
1,"PHS Reverie 8th, MSR Supply LLC",3.703704,8,0,27,29.0,70.981481
2,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000
3,State Flower - Jarred Flower - 3.5 gram - Gela...,3.964286,8,0,28,30.0,70.000000
4,budbud Black Gold OG 3.5g,4.086957,7,0,23,31.0,69.391304
5,Cartridge Live - Alien Labs - Half Gram AIO - ...,4.714286,12,0,14,17.0,46.435714
6,Kings Garden GMO Flower Prepackaged Quarter*,4.866667,10,0,15,16.0,95.000000
7,19023 OASIS Chocolate Peppermint Patties 100mg...,5.000000,8,1,19,21.0,24.802632
8,Source Flower Wookies Quarter 7g,5.000000,10,3,12,13.0,94.208333
9,19021 OASIS Chocolate Peanut Butter Cups Minis...,5.000000,8,1,23,24.0,25.886957


In [13]:
query_top_product_by_price = ifa_query.best_selling_products_by_liquidity(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'], SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, 'tx_product_name',
    'avg_sale_price', 'DESC', '10')
pd.read_sql_query(query_top_product_by_price, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,Source Flower Quest Half Ounce,19.196429,54,2,56,59.0,238.062500
1,Source Flower Motor Breath Half Ounce,25.900000,49,4,20,20.0,227.900000
2,16879 Union Electric Flower 28.0g Pouch Hybrid...,144.800000,208,50,15,16.0,212.400000
3,16881 Union Electric Flower 28.0g Pouch Sativa...,52.500000,86,7,16,16.0,209.062500
4,Almora Farm: 28g Flower: Legend OG [I],31.684211,51,7,19,19.0,206.210526
5,budbud Banana Cream Pie 14g,15.431373,37,0,51,56.0,186.372549
6,budbud Cosmic Glue 14g,39.690722,89,0,97,99.0,179.072165
7,budbud Blue Dream 14g,43.575758,86,7,33,38.0,178.030303
8,Day Dreams Pouch (Sativa) Crescendo 14g,30.507042,66,3,71,74.0,176.521127
9,SF-GDP-I-F-14.0,8.000000,15,3,11,11.0,175.000000


In [14]:
query_valuation_time_discount = ifa_query.create_company_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, .2)
pd.read_sql_query(query_valuation_time_discount, engine)

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,26.420454,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333
...,...,...,...,...,...,...,...
287,21.505694,8164573,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
288,21.505694,8164574,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
289,27.095957,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,35.010000
290,54.118077,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,69.333333


In [15]:
query_valuation_time_discount = ifa_query.create_company_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, .2, False)
pd.read_sql_query(query_valuation_time_discount, engine)

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,26.420454,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333
...,...,...,...,...,...,...,...
224,12.588183,7903512,Kushy Punch Recover Gummy,Edible (weight - each),4.0,1.504110,16.560000
225,12.676831,7903524,Kushy Punch Sativa Gummy,Edible (weight - each),4.0,1.660274,17.158261
226,27.095957,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,35.010000
227,54.118077,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,69.333333


In [16]:
query_join = ifa_query.create_inventory_days_since_package_metric_query_by_product_name(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE , SALES_TRANSACTIONS_END_DATE)
pd.read_sql_query(query_join, engine) 

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0152854172,active,Consumer,2021-12-23 18:43:34+00:00,2021-12-01 00:00:00+00:00,3,145.0,active,20358298,1A4060300022B79000204396,"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",Capsule (weight - each),Each,1.0,60.0,60.0,24704754,1A4060300022B79000319186,active,2022-04-28,2022-05-19 11:18:52+00:00,Product,"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",Capsule (weight - each),44.0,Each,2346868,Concentrate,,11722GELCAP25DUO,"TB-10.17.20-HV3009, TB-10.19.20-HV3020, TB-11....",False,False,False,None,None,-126,21
1,C9-0000370-LIC,0167152618,active,Consumer,2022-02-23 17:29:16+00:00,2022-02-01 00:00:00+00:00,1,65.0,active,20358292,1A4060300022B79000204410,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),Each,1.0,65.0,65.0,24704654,1A4060300022B79000319194,active,2022-04-28,2022-05-12 11:18:29+00:00,Product,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),20.0,Each,3205886,Concentrate,,13122LDSCAP25DUO,"Amnesia FF, Harvest FF, PY-09.30.20-AG0671, PY...",False,False,False,None,None,-64,14
2,C9-0000370-LIC,0105840520,active,Consumer,2021-06-06 20:39:13+00:00,2021-06-01 00:00:00+00:00,1,80.0,active,14804696,1A4060300022B79000068312,"THC Bomb Live Distillate 1g 510 Vape Cart (H),...",Vape Cartridge (weight - each),Each,2.0,80.0,40.0,24704664,1A4060300022B79000319188,active,2022-04-28,2022-05-05 11:18:15+00:00,Product,"THC Bomb Live Distillate 1g 510 Vape Cart (H),...",Vape Cartridge (weight - each),10.0,Each,2853854,Concentrate,,11822THCBLRD,"""33"" 12-12-19, #19, #20, #21, #22, #24, #25, #...",False,False,False,None,None,-326,7
3,C9-0000370-LIC,0078119533,active,Consumer,2021-02-05 12:53:16+00:00,2021-02-01 00:00:00+00:00,2,115.0,active,10957972,1A4060300022B79000015165,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",Capsule (weight - each),Each,1.0,55.0,55.0,24704670,1A4060300022B79000319184,active,2022-04-28,2022-05-18 11:18:46+00:00,Product,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",Capsule (weight - each),41.0,Each,2365799,Concentrate,,112421GELCAP50,"Amnesia FF, Clem FF, Clem FF1, CLEMFF, Harvest...",False,False,False,None,None,-447,20
4,C9-0000370-LIC,0134711863,active,Consumer,2021-10-07 17:30:35+00:00,2021-10-01 00:00:00+00:00,2,135.0,active,16988554,1A4060300022B79000127276,"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",Capsule (weight - each),Each,1.0,60.0,60.0,24704754,1A4060300022B79000319186,active,2022-04-28,2022-05-19 11:18:52+00:00,Product,"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",Capsule (weight - each),44.0,Each,2346868,Concentrate,,11722GELCAP25DUO,"TB-10.17.20-HV3009, TB-10.19.20-HV3020, TB-11....",False,False,False,None,None,-203,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6308,C9-0000370-LIC,0092188754,active,Consumer,2021-04-09 17:12:21+00:00,2021-04-01 00:00:00+00:00,5,86.0,active,9170655,1A4060300006E2E000052160,KE - THC NANO - Cran Pomegranate - 86g,Edible (weight - each),Each,1.0,20.0,20.0,7628335,1A4060300006E2E000042533,active,2020-09-01,2020-09-12 22:58:06+00:00,Product,KE - THC NANO - Cran Pomegranate - 86g,Edible (weight - each),0.0,Each,383489,InfusedEdible,,,"Banana Cream - 937.1-A 2/7/20 #30, Banana Crea...",False,False,False,None,None,220,11
6309,C9-0000370-LIC,0092188754,act

In [17]:
query_join = ifa_query.create_inventory_incoming_days_since_package_metric_query(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE , SALES_TRANSACTIONS_END_DATE)
pd.read_sql_query(query_join, engine)

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,inc_package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,inc_product_category_name,inc_product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,price_per_unit,package_id,package_label_1,type_1,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names_1,is_testing_sample_1,is_trade_sample_1,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003437071,2022-05-03,2022-05-17 01:53:18+00:00,C11-0001352-LIC,"HUM MADE, INC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24758908,1A4060300031895000003287,transfer,1A4060300031895000002309,HOOP 6_PANCAKE,Accepted,False,False,Flower (packaged gram - each),Humboldts Finest | Packaged Product | Demo | 1...,passed,0.13,13.0,Each,13.0,Each,0.13,1.0,Grams,0.01,24758908,1A4060300031895000003287,active,2022-05-02,2022-05-17 01:53:19+00:00,Product,Humboldts Finest | Packaged Product | Demo | 1...,Flower (packaged gram - each),13.0,Each,3650135,Buds,,,HOOP 6_PANCAKE,False,False,False,None,None,1,15
1,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003437071,2022-05-03,2022-05-17 01:53:18+00:00,C11-0001352-LIC,"HUM MADE, INC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24758817,1A4060300031895000003288,transfer,1A4060300031895000000239,"Wak h11 July 29, Wak h2 july 6",Accepted,False,False,Flower (packaged gram - each),Humboldts Finest | Packaged Product | Wakanda ...,passed,0.12,12.0,Each,12.0,Each,0.12,1.0,Grams,0.01,24758817,1A4060300031895000003288,active,2022-05-02,2022-05-17 01:53:19+00:00,Product,Humboldts Finest | Packaged Product | Wakanda ...,Flower (packaged gram - each),12.0,Each,3623725,Buds,,,"Wak h11 July 29, Wak h2 july 6",False,False,False,None,None,1,15
2,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003422531,2022-04-29,2022-04-30 02:29:13+00:00,C11-0001274-LIC,"NABITWO, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24704754,1A4060300022B79000319186,transfer,1A4060300005598000014849,"TB-10.17.20-HV3009, TB-10.19.20-HV3020, TB-11....",Accepted,False,False,Capsule (weight - each),"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",passed,1008.00,48.0,Each,48.0,Each,1008.00,28.0,Grams,21.00,24704754,1A4060300022B79000319186,active,2022-04-28,2022-05-19 11:18:52+00:00,Product,"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",Capsule (weight - each),44.0,Each,2346868,Concentrate,,11722GELCAP25DUO,"TB-10.17.20-HV3009, TB-10.19.20-HV3020, TB-11....",False,False,False,None,None,1,21
3,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003422531,2022-04-29,2022-04-30 02:29:13+00:00,C11-0001274-LIC,"NABITWO, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24704609,1A4060300022B79000319210,transfer,1A4060300005598000014127,"1-21, 2020-10-06 Jack Harvest Greenhouse 13, 5...",Accepted,False,False,Vape Cartridge (weight - each),Blue Banana Live Distillate 1g 510 Vape Cart (...,passed,408.00,24.0,Each,24.0,Each,408.00,1.0,Grams,17.00,24704609,1A4060300022B79000319210,active,2022-04-28,2022-04-30 02:29:14+00:00,Product,Blue Banana Live Distillate 1g 510 Vape Cart (...,Vape Cartridge (weight - each),24.0,Each,2824323,Concentrate,,31122BLUBANLRD,"1-21, 2020-10-06 Jack Harvest Greenhouse 13, 5...",False,False,False,None,None,1,2
4,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003422531,2022-04-29,2022-04-30 02:29:13+00:00,C11-0001274-LIC,"NABITWO, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24704746,1A4060300022B79000319197,

In [18]:
query_join = ifa_query.create_inventory_days_since_package_metric_query(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE)
pd.read_sql_query(query_join, engine)

,license_number,receipt_number,rt_type,sales_customer_type,sales_datetime,sales_month,total_packages,rt_total_price,tx_type,tx_package_id,tx_package_label,tx_product_name,tx_product_category_name,tx_unit_of_measure,tx_quantity_sold,tx_total_price,price_per_unit,package_id,package_label,type,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names,is_testing_sample,is_trade_sample,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,C9-0000370-LIC,0187745893,active,Consumer,2022-05-15 20:41:38+00:00,2022-05-01 00:00:00+00:00,1,165.0,active,24704670,1A4060300022B79000319184,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",Capsule (weight - each),Each,3.0,165.0,55.0,24704670,1A4060300022B79000319184,active,2022-04-28,2022-05-18 11:18:46+00:00,Product,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",Capsule (weight - each),41.0,Each,2365799,Concentrate,,112421GELCAP50,"Amnesia FF, Clem FF, Clem FF1, CLEMFF, Harvest...",False,False,False,None,None,17,20
1,C9-0000370-LIC,0185025683,active,Consumer,2022-05-04 15:06:46+00:00,2022-05-01 00:00:00+00:00,2,70.0,active,24704577,1A4060300022B79000320205,"BBrand Pineapple 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704577,1A4060300022B79000320205,active,2022-04-28,2022-05-14 11:19:20+00:00,Product,"BBrand Pineapple 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),10.0,Each,3389932,Concentrate,,40522BDTPIN,"01/10/21, 10KGREENHOUSE_CHILI VERDE, 10KGREENH...",False,False,False,None,None,6,16
2,C9-0000370-LIC,0185025683,active,Consumer,2022-05-04 15:06:46+00:00,2022-05-01 00:00:00+00:00,2,70.0,active,24704603,1A4060300022B79000320206,"BBrand Raspberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),Each,1.0,35.0,35.0,24704603,1A4060300022B79000320206,active,2022-04-28,2022-05-09 11:18:37+00:00,Product,"BBrand Raspberry 1g Vape, Hashtag Distribution...",Vape Cartridge (weight - each),9.0,Each,3389930,Concentrate,,41222BDTRAS,"0649_SFV OG 12-07, 10-15 peanut butter souffle...",False,False,False,None,None,6,11
3,C9-0000370-LIC,0184086989,active,Consumer,2022-04-30 16:29:57+00:00,2022-04-01 00:00:00+00:00,1,65.0,active,24704654,1A4060300022B79000319194,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),Each,1.0,65.0,65.0,24704654,1A4060300022B79000319194,active,2022-04-28,2022-05-12 11:18:29+00:00,Product,"Buddies LD 25mg Sativa Capsule 40pc, Hashtag D...",Capsule (weight - each),20.0,Each,3205886,Concentrate,,13122LDSCAP25DUO,"Amnesia FF, Harvest FF, PY-09.30.20-AG0671, PY...",False,False,False,None,None,2,14
4,C9-0000370-LIC,0185025665,active,Consumer,2022-05-04 13:41:47+00:00,2022-05-01 00:00:00+00:00,2,75.0,active,24704734,1A4060300022B79000319200,"Buddies THC 10mg Capsule 60pc, Hashtag Distrib...",Capsule (weight - each),Each,1.0,50.0,50.0,24704734,1A4060300022B79000319200,active,2022-04-28,2022-05-05 11:18:17+00:00,Product,"Buddies THC 10mg Capsule 60pc, Hashtag Distrib...",Capsule (weight - each),35.0,Each,2379103,Concentrate,,22522GELCAP10,"Amnesia FF, Clem FF, Clem FF1, CLEMFF, Harvest...",False,False,False,None,None,6,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2011,C9-0000370-LIC,0069096734,active,Consumer,2020-12-24 13:49:38+00:00,2020-12-01 00:00:00+00:00,4,100.0,active,7903524,1A406030000A2C9000007835,Kushy Punch Sativa Gummy,Edible (weight - each),Each,1.0,20.0,20.0,7903524,1A406030000A2C9000007835,active,2020-09-14,2022-05-13 11:18:37+00:00,Product,Kushy Punch Sativa Gummy,Edible (weight - each),4.0,Each,1329552,InfusedEdible,,,"2019-08-10-Barn Doobland-H, 2019-09-20-Barn Do...",False,False,False,None,None,101,606
2012,C9-0000370-LIC,0120152802,active,Consumer,2021-08-06 19:36:31+00:0

In [19]:
query_metric = ifa_query.create_company_sale_metric_query(
    ['EL', '99HT'],
    ['C9-0000370-LIC', 'C10-0000279-LIC'], SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, 'tx_product_name')
pd.read_sql_query(query_metric, engine)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,"Buddies THC 25mg Capsule 40pc, Hashtag Distrib...",27.135593,78,1,59,64.0,60.000000
1,Surplus Purified Live Resin: 1g Cart: Frozen M...,39.918367,89,5,49,51.0,44.102041
2,Hush - Distillate Cart - Super Lemon Haze - 1g,42.110000,61,7,100,115.0,34.965000
3,Brite - |CART| Full Spectrum (0.6 g) 510 Cartr...,41.400000,70,11,15,19.0,25.000000
4,HF | DEP | 8TH | 3.5G | WAKANDA BAG,33.031250,67,3,32,65.0,38.781250
...,...,...,...,...,...,...,...
2782,"Cresco Mindy’s 1:1 Honey Sweet Melon 40mg, 20 pcs",100.000000,176,1,10,12.0,14.250000
2783,STIIIZY - Super Green Crack CDT - 0.5g,15.000000,26,4,2,5.0,40.000000
2784,STIIIZY - Space Glue Liquid LRP - 0.5g,48.200000,63,20,5,5.0,40.000000
2785,"Good News Friyay Gummy 100mg, 10pc Pouch SAMPLE",30.500000,47,14,2,2.0,1.000000


In [20]:
query_join = ifa_query.create_inventory_incoming_days_since_package_metric_query_by_product_name(
    ['EL'], ['C9-0000370-LIC'], True, SALES_TRANSACTIONS_START_DATE , SALES_TRANSACTIONS_END_DATE)
pd.read_sql_query(query_join, engine)

,delivery_type,license_number,manifest_number,created_date,received_datetime,shipper_facility_license_number,shipper_facility_name,recipient_facility_license_number,recipient_facility_name,shipment_type_name,inc_package_id,package_label,type,source_package_labels,source_harvest_names,shipment_package_state,is_testing_sample,is_trade_sample,inc_product_category_name,inc_product_name,package_lab_results_status,shipper_wholesale_price,shipped_quantity,shipped_unit_of_measure,received_quantity,received_unit_of_measure,receiver_wholesale_price,item_unit_weight,item_unit_weight_unit_of_measure_name,price_per_unit,package_id,package_label_1,type_1,packaged_date,last_modified_at,package_type,product_name,product_category_name,quantity,unit_of_measure,item_id,item_product_category_type,production_batch_number,source_production_batch_numbers,source_harvest_names_1,is_testing_sample_1,is_trade_sample_1,is_on_hold,archived_date,finished_date,sales_days_diff,inventory_days_diff
0,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003437071,2022-05-03,2022-05-17 01:53:18+00:00,C11-0001352-LIC,"HUM MADE, INC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24758908,1A4060300031895000003287,transfer,1A4060300031895000002309,HOOP 6_PANCAKE,Accepted,False,False,Flower (packaged gram - each),Humboldts Finest | Packaged Product | Demo | 1...,passed,0.13,13.0,Each,13.0,Each,0.13,1.0,Grams,0.01,24758908,1A4060300031895000003287,active,2022-05-02,2022-05-17 01:53:19+00:00,Product,Humboldts Finest | Packaged Product | Demo | 1...,Flower (packaged gram - each),13.0,Each,3650135,Buds,,,HOOP 6_PANCAKE,False,False,False,None,None,1,15
1,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003437071,2022-05-03,2022-05-17 01:53:18+00:00,C11-0001352-LIC,"HUM MADE, INC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24758817,1A4060300031895000003288,transfer,1A4060300031895000000239,"Wak h11 July 29, Wak h2 july 6",Accepted,False,False,Flower (packaged gram - each),Humboldts Finest | Packaged Product | Wakanda ...,passed,0.12,12.0,Each,12.0,Each,0.12,1.0,Grams,0.01,24758817,1A4060300031895000003288,active,2022-05-02,2022-05-17 01:53:19+00:00,Product,Humboldts Finest | Packaged Product | Wakanda ...,Flower (packaged gram - each),12.0,Each,3623725,Buds,,,"Wak h11 July 29, Wak h2 july 6",False,False,False,None,None,1,15
2,INCOMING_FROM_VENDOR,C9-0000370-LIC,0002811313,2021-12-09,2021-12-10 00:16:43+00:00,C11-0001274-LIC,"NABITWO, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,20358293,1A4060300022B79000204408,transfer,1A4060300005598000010328,"TB-10.17.20-HV3009, TB-10.17.20-HV3010, TB-10....",Accepted,False,False,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",passed,504.00,24.0,Each,24.0,Each,504.00,14.0,Grams,21.00,24704670,1A4060300022B79000319184,active,2022-04-28,2022-05-18 11:18:46+00:00,Product,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",Capsule (weight - each),41.0,Each,2365799,Concentrate,,112421GELCAP50,"Amnesia FF, Clem FF, Clem FF1, CLEMFF, Harvest...",False,False,False,None,None,-140,20
3,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003422432,2022-04-29,2022-04-30 01:53:39+00:00,C11-0001274-LIC,"NABITWO, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24704709,1A4060300022B79000320200,transfer,1A4060300005598000014033,"DSD-10.15.20-HV1317, DSD-11.20.20-HV1341, DSD-...",Accepted,False,False,Capsule (weight - each),"Buddies THC 50mg Capsule 2pc, Hashtag Distribu...",passed,240.00,120.0,Each,120.0,Each,240.00,600.0,Milligrams,2.00,24704709,1A4060300022B79000320200,active,2022-04-28,2022-05-09 11:18:37+00:00,Product,"Buddies THC 50mg Capsule 2pc, Hashtag Distribu...",Capsule (weight - each),119.0,Each,4099941,Concentrate,,30322GELCAP50,"DSD-10.15.20-HV1317, DSD-11.20.20-HV1341, DSD-...",False,False,False,None,None,1,11
4,INCOMING_FROM_VENDOR,C9-0000370-LIC,0003422531,2022-04-29,2022-04-30 02:29:13+00:00,C11-0001274-LIC,"NABITWO, LLC",C9-0000370-LIC,"ELITE HERBS, INC",Wholesale Manifest,24704756,1A4060300022B79000319183,tr

In [21]:
query_valuation_time_discount = ifa_query.create_company_incoming_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, .2, False)
pd.read_sql_query(query_valuation_time_discount, engine)

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,12.070481,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.00
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.00
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.00
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.00
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.00
...,...,...,...,...,...,...,...
363,0.009484,8165921,Honeyleaf Cartridge - OG Kush - 0.5g,Vape Cartridge (volume - each),0.0,0.290411,0.01
364,13.931083,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,18.00
365,39.027460,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,50.00
366,19.709653,9166120,N5 - Harmony 15ml - 15ml,Tincture (volume - each),2.0,1.304110,25.00


In [22]:
query_valuation_time_discount = ifa_query.create_company_incoming_inventory_valudation_by_discounting_time_value(
    ['EL'],
    ['C9-0000370-LIC'],
    True, SALES_TRANSACTIONS_START_DATE, SALES_TRANSACTIONS_END_DATE, .2)
pd.read_sql_query(query_valuation_time_discount, engine)

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,12.070481,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.000
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.000
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.000
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.000
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.000
...,...,...,...,...,...,...,...
363,0.009484,8165921,Honeyleaf Cartridge - OG Kush - 0.5g,Vape Cartridge (volume - each),0.0,0.290411,0.010
364,13.931083,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,18.000
365,39.027460,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,50.000
366,9.858769,9166120,N5 - Harmony 15ml - 15ml,Tincture (volume - each),2.0,1.304110,12.505


# Analyzer

In [23]:
analyzer = ifa.Analyzer()

In [24]:
analyzer.update_class_attributes(['EL'], ['C9-0000370-LIC'], True)

In [25]:
analyzer.find_most_valuable_products('tx_product_name', 'avg_days_since_sale', 'ASC', 10)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000
1,Kings Garden GMO Flower Prepackaged Quarter*,4.866667,10,0,15,16.0,95.000000
2,Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000
3,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000
4,XJ-13 1/4 oz Jar,6.812500,22,1,16,16.0,80.000000
5,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667
6,Flower Jars - Peanut Butter Breath - 3.5g,7.181818,15,0,22,28.0,42.795455
7,Wedding Cake LR Liquid Diamonds 1g 510 Vape Ca...,7.583333,15,1,12,12.0,45.000000
8,Kings Garden Cake Walk Live Budder (1g),8.250000,14,0,12,12.0,35.000000
9,HUMBOLDT TREES - Pre Roll Single - 0.7g - Tropaya,8.576923,17,1,26,32.0,9.961538


In [26]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Surplus: 1g Cart: Blueberry Cookies [I],27.264706,59,5,136,145.0,39.911765,4.988134,5.318231
1,Surplus: 1g Cart: Trainwreck [H],32.243750,92,3,160,174.0,39.750000,4.962202,5.396395
2,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
3,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
4,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410
5,Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000,4.000000,4.000000
6,Surplus: 1g Cart: Purple Lemonade [S],29.333333,70,3,108,125.0,40.000000,3.681818,4.261364
7,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.602564,51,1,78,91.0,55.000000,3.610682,4.212463
8,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
9,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,1.000000,3.494545,3.607273


In [27]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15, True)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Surplus: 1g Cart: Trainwreck [H],32.243750,92,3,160,174.0,39.750000,4.962202,5.396395
2,Surplus: 1g Cart: Blueberry Cookies [I],27.264706,59,5,136,145.0,39.911765,4.988134,5.318231
3,HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
4,Surplus: 1g Cart: Green Crack [S],39.184615,87,4,130,190.0,39.122308,3.317629,4.848842
5,Kings Garden GMO Flower Prepackaged Gram*,10.041667,26,1,24,48.0,16.995833,2.390041,4.780083
6,Kings Garden Wedding Gushers Flower Prepackage...,14.225806,36,1,31,68.0,15.379032,2.179138,4.780045
7,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
8,Surplus: 1g Cart: Purple Lemonade [S],29.333333,70,3,108,125.0,40.000000,3.681818,4.261364
9,"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.602564,51,1,78,91.0,55.000000,3.610682,4.212463


In [28]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],27.264706,59,5,136,145.0,39.911765,4.988134,5.318231
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],32.243750,92,3,160,174.0,39.750000,4.962202,5.396395
2,Pre-Roll Leaf,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
3,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
4,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410
5,Flower (packaged half ounce - each),Lifted Organics American Weed Sour Strawberry ...,6.000000,13,0,24,24.0,130.000000,4.000000,4.000000
6,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.602564,51,1,78,91.0,55.000000,3.610682,4.212463
7,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
8,Pre-Roll Flower,Calm 108-1g Pre-roll,8.870968,21,2,31,32.0,1.000000,3.494545,3.607273
9,Pre-Roll Flower,1.0g Cone - Mendo Breath,35.264463,169,1,121,125.0,6.113388,3.431216,3.544645


In [29]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name', 15, True)

,tx_product_category_name,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Pre-Roll Flower,Highrize Sour Patch Kidz PF .7g,17.402778,35,1,72,175.0,4.986111,4.137271,10.055866
1,Vape Cartridge (volume - each),Surplus: 1g Cart: Trainwreck [H],32.243750,92,3,160,174.0,39.750000,4.962202,5.396395
2,Vape Cartridge (volume - each),Surplus: 1g Cart: Blueberry Cookies [I],27.264706,59,5,136,145.0,39.911765,4.988134,5.318231
3,Flower (packaged eighth - each),HUMBOLDT TREES - Flower - LITTLE TREES - 3.5g ...,3.928571,7,0,14,20.0,30.000000,3.563636,5.090909
4,Vape Cartridge (volume - each),Surplus: 1g Cart: Green Crack [S],39.184615,87,4,130,190.0,39.122308,3.317629,4.848842
5,Flower (packaged gram - each),Kings Garden GMO Flower Prepackaged Gram*,10.041667,26,1,24,48.0,16.995833,2.390041,4.780083
6,Flower (packaged gram - each),Kings Garden Wedding Gushers Flower Prepackage...,14.225806,36,1,31,68.0,15.379032,2.179138,4.780045
7,Pre-Roll Leaf,Peach Ozz-Baker's 1g Pre-roll,6.966667,22,2,30,30.0,0.996667,4.306220,4.306220
8,Capsule (weight - each),"Buddies THC 50mg Capsule 20pc, Hashtag Distrib...",21.602564,51,1,78,91.0,55.000000,3.610682,4.212463
9,Pre-Roll Flower,Highrize - OG Kush Premium Flower Single Tubes,6.240000,10,2,25,25.0,1.000000,4.006410,4.006410


In [30]:
df_v1 = analyzer.get_inventory_valuation_query('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank
0,11497141,25.60,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1
...,...,...,...,...,...,...,...,...
287,8164573,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
288,8164574,21.00,0.0,BIIIT - Sour Straws Assorted,Edible (weight - each),0.320548,0.00,1
289,8486461,23.04,4.0,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),1.405479,92.16,1
290,9166114,64.00,1.0,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.358904,64.00,1


In [31]:
df_v2 = analyzer.get_inventory_valuation_query('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit
0,26.420454,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333
...,...,...,...,...,...,...,...
287,21.505694,8164573,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
288,21.505694,8164574,BIIIT - Sour Straws Assorted,Edible (weight - each),0.0,0.320548,22.800000
289,27.095957,8486461,Fiori Friend CBD Pet Supplement (300mg CBD),Tincture (volume - each),4.0,1.405479,35.010000
290,54.118077,9166114,N5 - Harmony 30ml - 30ml,Tincture (volume - each),1.0,1.358904,69.333333


In [32]:
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,11497141,0.00,8.0,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),1.191781,204.80,1,Concentrates,1.0,True
1,12801666,20.48,34.0,Level Protab Hangover,Extract (weight - each),0.846575,696.32,1,Concentrates,1.0,False
2,12801667,12.80,8.0,Petra Mints Pineapple - 100mg,Edible (weight - each),0.849315,102.40,1,Edibles,1.0,False
3,12801668,22.40,36.0,Level Tablingual Calm CBG,Extract (weight - each),0.953425,806.40,1,Concentrates,1.0,False
4,12801670,19.20,33.0,Level Tablingual Sleep CBN,Extract (weight - each),0.934247,633.60,1,Concentrates,1.0,False


In [33]:
valuation_1

103200.48000000001

In [34]:
df_v1.shelf_life.value_counts()

1.0    251
0.5     41
Name: shelf_life, dtype: int64

In [35]:
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727,Concentrates,1.0,True
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333,Concentrates,1.0,False
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667,Edibles,1.0,False
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000,Concentrates,1.0,False
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333,Concentrates,1.0,False


In [36]:
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,32.832727,Concentrates,1.0,True
1,30.348268,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,35.413333,Concentrates,1.0,False
2,16.399981,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,19.146667,Edibles,1.0,False
3,20.973177,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,24.955000,Concentrates,1.0,False
4,21.585000,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,25.593333,Concentrates,1.0,False


In [37]:
valuation_2

100441.5866137401

In [38]:
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.0,Concentrates,1.0,True
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.0,Concentrates,1.0,False
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.0,Edibles,1.0,False
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.0,Concentrates,1.0,False
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.0,Concentrates,1.0,False


In [39]:
valuation

48052.385328723685

In [40]:
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', False, discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,11497141,Sugar Cone - |GLAZE| Applicator (1.0 g) Glazed...,Other Concentrate (weight - each),8.0,1.191781,15.0,Concentrates,1.0,True
1,11.140648,12801666,Level Protab Hangover,Extract (weight - each),34.0,0.846575,13.0,Concentrates,1.0,False
2,6.852360,12801667,Petra Mints Pineapple - 100mg,Edible (weight - each),8.0,0.849315,8.0,Edibles,1.0,False
3,10.085278,12801668,Level Tablingual Calm CBG,Extract (weight - each),36.0,0.953425,12.0,Concentrates,1.0,False
4,10.120604,12801670,Level Tablingual Sleep CBN,Extract (weight - each),33.0,0.934247,12.0,Concentrates,1.0,False


In [41]:
valuation

48462.60442681242

# Test many companies

In [42]:
COMPANY_IDENTIFIER_LIST = [
    'DL',
    'DW',
    'EMA',
    'EMM',
    'EMT',
    'EMF',
    'ST',
    'GRG',
    'EL',
    'VS',
]
license_numbers = ['C10-0000169-LIC', '402R-00545', '402-00840', 'C9-0000323-LIC',
       'MR282376', 'C10-0000695-LIC', 'C9-0000370-LIC', 'C10-0000774-LIC',
       'C10-0000786-LIC', 'MR283369', 'C10-0000408-LIC',
       'C10-0000939-LIC']

In [43]:
analyzer.update_class_attributes(COMPANY_IDENTIFIER_LIST, license_numbers, True, '2020-01-01', '2022-09-01')

In [44]:
analyzer.find_most_valuable_products('tx_product_name', 'avg_days_since_sale', 'ASC', 10)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price
0,Wax - Mimosa Cake - KM,0.344828,1,0,29,45.00,16.503391
1,Powder Cakes,0.533333,1,0,15,125.73,3.716691
2,Wax - Banana Bliss,0.956522,1,0,23,30.00,19.514783
3,Wax - Banana Kush 1G,1.000000,1,1,32,40.00,16.720000
4,Cookies N Cream | Live Rosin | Grammed,1.000000,1,1,16,20.00,32.698750
5,Wax - Pie Tai - SUN,1.033333,2,1,30,45.00,16.677278
6,The Sweeties #7 | Bud,1.083333,2,1,12,168.00,8.073274
7,Strawberry Slurpee | Live Rosin | Gramed,1.090909,4,0,11,15.00,30.117273
8,Wax - Dragon Milk - KM,1.114286,3,1,35,45.00,16.958429
9,Karma - Krispie - 100MG - Original,1.147727,2,0,88,156.00,2.000000


In [45]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_name', 15)

,tx_product_name,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Bootylicious | 3.5g,6.665259,27,1,2605,10507.0,15.176665,390.832517,1576.382826
1,Black Mamba | 3.5g,9.394603,26,0,3150,12344.5,14.810910,335.298888,1313.999088
2,Stardawg | 3.5g,15.193587,41,0,5052,21437.5,14.476205,332.508716,1410.957164
3,Bio-Jesus 8ths (3.5g Flower),3.258065,9,1,1023,4025.0,15.677419,313.990099,1235.396040
4,Lemon Meringue | 3.5g,9.495083,45,0,2949,11805.5,14.412192,310.581801,1243.327720
5,Afghani Purps | 3.5g,13.949536,51,1,3983,16999.5,13.332674,285.529220,1218.642726
6,0.5g Pre Roll - Meltdown,1.343008,3,0,379,250.0,19.788918,282.202358,186.149312
7,Citrique | 3.5g,6.428734,21,1,1761,6779.5,15.107550,273.926420,1054.562274
8,East Coast Sour Diesel 3.5 g Strane,2.399386,9,0,651,2558.5,14.602989,271.319462,1066.314661
9,"CL,Preroll,(H)Lemon Lava,,,1g",3.009284,9,1,754,897.0,15.928899,250.557955,298.077567


In [46]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, license_number', 15)

,tx_product_category_name,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Buds,402R-00545,18.539150,437,0,164906,1087618.581,7.161937,8895.014268,58666.044876
1,Concentrate,402R-00545,17.489659,507,0,107828,147086.010,24.814811,6165.242969,8409.884158
2,Vape Cartridge (weight - each),C10-0000169-LIC,19.673849,378,0,112687,125369.000,27.637025,5727.755719,6372.367769
3,Raw Pre-Rolls,MR282376,30.359287,241,0,143707,182908.570,15.386717,4733.543376,6024.798026
4,Concentrate (Each),402R-00545,19.420562,434,0,83184,95081.100,28.690639,4283.295278,4895.898570
5,Buds,MR282376,33.452002,231,0,133871,521070.100,14.229775,4001.883020,15576.649054
6,Infused (edible),402R-00545,22.680010,303,0,84887,106891.000,15.416179,3742.811418,4713.004998
7,Flower (packaged eighth - each),C10-0000169-LIC,35.297270,381,0,108504,119368.000,36.461283,3074.005427,3381.791269
8,Edible (weight - each),C10-0000695-LIC,30.048169,277,0,72162,80553.000,19.946745,2401.543969,2680.795590
9,Edible (weight - each),C10-0000169-LIC,24.485424,303,0,56223,74969.000,13.449356,2296.182404,3061.780741


In [47]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, license_number', 15, True)

,tx_product_category_name,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Buds,402R-00545,18.539150,437,0,164906,1.087619e+06,7.161937,8895.014268,58666.044876
1,Buds,MR282376,33.452002,231,0,133871,5.210701e+05,14.229775,4001.883020,15576.649054
2,Concentrate,402R-00545,17.489659,507,0,107828,1.470860e+05,24.814811,6165.242969,8409.884158
3,Flower,C10-0000169-LIC,14.502803,250,0,22301,9.392767e+04,10.037094,1537.702792,6476.518568
4,Vape Cartridge (weight - each),C10-0000169-LIC,19.673849,378,0,112687,1.253690e+05,27.637025,5727.755719,6372.367769
5,Raw Pre-Rolls,MR282376,30.359287,241,0,143707,1.829086e+05,15.386717,4733.543376,6024.798026
6,Concentrate (Each),402R-00545,19.420562,434,0,83184,9.508110e+04,28.690639,4283.295278,4895.898570
7,Infused (edible),402R-00545,22.680010,303,0,84887,1.068910e+05,15.416179,3742.811418,4713.004998
8,Buds,402-00840,27.926150,236,0,10237,1.068535e+05,4.790844,366.573979,3826.287407
9,Flower (packaged eighth - each),C10-0000169-LIC,35.297270,381,0,108504,1.193680e+05,36.461283,3074.005427,3381.791269


In [48]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name, license_number', 15)

,tx_product_category_name,tx_product_name,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Flower (packaged ounce - each),House Party - Alien OG - Flower - Littles,C10-0000408-LIC,6.157528,81,0,4450,4451.0,42.560899,722.692602,722.855005
1,Buds,Bootylicious | 3.5g,MR282376,6.665259,27,1,2605,10507.0,15.176665,390.832517,1576.382826
2,Buds,Black Mamba | 3.5g,MR282376,9.394603,26,0,3150,12344.5,14.810910,335.298888,1313.999088
3,Buds,Stardawg | 3.5g,MR282376,15.189744,41,0,4875,20741.0,14.473292,320.940243,1365.460837
4,Buds,Bio-Jesus 8ths (3.5g Flower),MR282376,3.258065,9,1,1023,4025.0,15.677419,313.990099,1235.396040
5,Buds,Lemon Meringue | 3.5g,MR282376,9.495083,45,0,2949,11805.5,14.412192,310.581801,1243.327720
6,Raw Pre-Rolls,0.5g Pre Roll - Meltdown,MR282376,1.343008,3,0,379,250.0,19.788918,282.202358,186.149312
7,Buds,Afghani Purps | 3.5g,MR282376,13.867876,51,1,3860,16534.0,13.258412,278.341117,1192.251821
8,Buds,Citrique | 3.5g,MR282376,6.428734,21,1,1761,6779.5,15.107550,273.926420,1054.562274
9,Buds,East Coast Sour Diesel 3.5 g Strane,MR282376,2.399386,9,0,651,2558.5,14.602989,271.319462,1066.314661


In [49]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name, license_number', 15, True)

,tx_product_category_name,tx_product_name,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Buds,Bootylicious | 3.5g,MR282376,6.665259,27,1,2605,10507.00,15.176665,390.832517,1576.382826
1,Buds,Stardawg | 3.5g,MR282376,15.189744,41,0,4875,20741.00,14.473292,320.940243,1365.460837
2,Buds,Dragon Glue,402R-00545,27.731626,111,4,4218,36821.53,6.062476,152.100708,1327.781123
3,Buds,Black Mamba | 3.5g,MR282376,9.394603,26,0,3150,12344.50,14.810910,335.298888,1313.999088
4,Buds,Lemon Meringue | 3.5g,MR282376,9.495083,45,0,2949,11805.50,14.412192,310.581801,1243.327720
5,Buds,Bio-Jesus 8ths (3.5g Flower),MR282376,3.258065,9,1,1023,4025.00,15.677419,313.990099,1235.396040
6,Buds,Afghani Purps | 3.5g,MR282376,13.867876,51,1,3860,16534.00,13.258412,278.341117,1192.251821
7,Buds,East Coast Sour Diesel 3.5 g Strane,MR282376,2.399386,9,0,651,2558.50,14.602989,271.319462,1066.314661
8,Buds,Citrique | 3.5g,MR282376,6.428734,21,1,1761,6779.50,15.107550,273.926420,1054.562274
9,Buds,Bruce Banner | 3.5g,MR282376,7.492143,21,0,1400,6475.00,12.731880,186.862427,864.238726


In [50]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('license_number', 15)

,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,402R-00545,20.867386,714,0,464927,1.477671e+06,16.821818,22280.078980,70812.449629
1,C10-0000169-LIC,27.392397,771,0,559862,7.146010e+05,27.489597,20438.591270,26087.568313
2,MR282376,37.604389,349,0,448921,9.490397e+05,22.454990,11937.994730,25237.472019
3,C10-0000695-LIC,34.537134,338,0,248489,2.890940e+05,28.092574,7194.835473,8370.527540
4,C10-0000408-LIC,27.510192,321,0,192310,2.421595e+05,22.462230,6990.500134,8802.537659
5,C9-0000323-LIC,27.366187,423,0,111039,1.798594e+05,32.158388,4057.525493,6572.320881
6,C10-0000786-LIC,38.447796,414,0,147614,1.684890e+05,26.526151,3839.335782,4382.279774
7,C10-0000774-LIC,35.180378,401,0,131607,1.497010e+05,24.794081,3740.920584,4255.241380
8,MR283369,43.190268,322,0,83645,1.867348e+05,18.679468,1936.663121,4323.539235
9,402-00840,35.433626,331,0,25861,1.289063e+05,16.850332,729.843456,3637.964607


In [51]:
# Also try incoming join with sales to find this information for all products 
# TODO filter by time horizon for sales or inventory
# Investigate avg_days_since_sale 
# Inventory valuation with incoming data

In [52]:
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,10099604,0.0,0.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.758904,0.00,1,Flower,0.5,True
1,10101288,0.0,20.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.917808,0.20,1,Flower,0.5,True
2,10101795,0.0,10.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,1.068493,0.10,1,Flower,0.5,True
3,10525023,23.5,1.0,Fuzzies - Naked Mini Live Resin - Packaged - K...,Pre-Roll Infused,0.276712,23.50,1,Flower,0.5,False
4,10570444,0.0,2.0,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),1.260274,17.64,1,Edibles,1.0,True


In [53]:
valuation_1

6240658.646366466

In [54]:
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,10099604,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.0,0.758904,2.483604,Flower,0.5,True
1,0.000000,10101288,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,20.0,0.917808,2.483604,Flower,0.5,True
2,0.000000,10101795,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,10.0,1.068493,2.483604,Flower,0.5,True
3,31.072346,10525023,Fuzzies - Naked Mini Live Resin - Packaged - K...,Pre-Roll Infused,1.0,0.276712,32.680182,Flower,0.5,False
4,0.000000,10570444,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),2.0,1.260274,9.571596,Edibles,1.0,True


In [55]:
valuation_2

6247235.843714759

# Different timeframe

In [56]:
analyzer.update_class_attributes(COMPANY_IDENTIFIER_LIST, license_numbers, True, '2021-01-01', '2022-01-01')

In [57]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, license_number', 15)

,tx_product_category_name,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Buds,402R-00545,17.085240,437,0,69545,434562.89,7.339736,4070.472579,25434.989253
1,Vape Cartridge (weight - each),C10-0000169-LIC,19.913794,317,0,54544,60694.00,28.477761,2739.005887,3047.837036
2,Raw Pre-Rolls,MR282376,32.245847,241,0,81087,105938.94,15.231114,2514.649402,3285.351438
3,Concentrate (Each),402R-00545,16.711958,434,0,40501,46782.50,27.743297,2423.474296,2799.342887
4,Concentrate,402R-00545,17.349845,507,0,41950,57210.82,26.246723,2417.889028,3297.483051
5,Buds,MR282376,33.913456,191,0,64083,255142.50,14.239912,1889.603938,7523.341181
6,Flower (packaged eighth - each),C10-0000169-LIC,31.706414,381,0,59383,65264.00,36.841339,1872.901787,2058.384761
7,Infused (edible),402R-00545,20.653291,242,0,38528,48891.00,14.886148,1865.465402,2367.225627
8,Edible (weight - each),C10-0000695-LIC,32.712347,277,0,35508,39926.00,19.587089,1085.461723,1220.517763
9,Edible (weight - each),C10-0000169-LIC,27.311345,303,0,25271,34342.00,13.389486,925.293133,1257.426171


In [58]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('tx_product_category_name, tx_product_name, license_number', 15)

,tx_product_category_name,tx_product_name,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,Flower (packaged ounce - each),House Party - Alien OG - Flower - Littles,C10-0000408-LIC,4.466667,42,0,2070,2070.00,39.543961,463.432836,463.432836
1,Flower (packaged half ounce - each),Mr. Nice Guy - Alien OG - Flower - Littles,C10-0000408-LIC,6.035361,70,0,1923,2339.00,28.032588,318.622178,387.549285
2,Flower (packaged quarter - each),14171 Pacific Stone Flower 7.0g Pouch Indica W...,C9-0000323-LIC,8.500981,43,1,1529,2003.00,35.461661,179.861594,235.619865
3,Flower (packaged half ounce - each),Mr. Nice Guy - Alien OG - Flower,C10-0000408-LIC,7.912070,28,0,1251,1475.00,43.170663,158.112851,186.424025
4,Buds,M00000646905: Purple Pineapple Express #4 | 3.5g,MR282376,10.014989,48,1,1401,5691.00,14.683010,139.890314,568.248236
5,Raw Pre-Rolls,M00000483522: GG #4 0.5 g Pre Roll Strane,MR282376,0.925373,3,0,134,99.50,19.910448,134.000000,134.000000
6,Buds,Stardawg | 3.5g,MR282376,14.233666,41,1,1806,7805.00,14.755308,126.882284,548.347857
7,Buds,Dragon Glue,402R-00545,26.278580,100,4,3324,28068.85,6.133680,126.490853,1068.126587
8,Raw Pre-Rolls,M00000238713: Sira MTF Preroll 1g,MR282376,2.831804,6,1,327,397.00,15.794495,115.474082,140.193305
9,Flower (packaged half ounce - each),Mr. Nice Guy - Chem Reserve - Flower - Littles,C10-0000408-LIC,12.695836,35,1,1417,1689.00,27.576570,111.611395,133.035742


In [59]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('license_number', 15)

,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,402R-00545,19.454344,648,0,199789,605898.41,17.085240,10269.634205,31144.632768
1,C10-0000169-LIC,28.164888,509,0,268339,308263.30,29.078402,9527.429890,10944.950150
2,MR282376,41.379491,349,0,243215,516438.49,22.574598,5877.670116,12480.542234
3,C10-0000695-LIC,38.421989,338,0,123046,135599.21,27.606163,3202.489114,3529.208539
4,C10-0000408-LIC,28.716933,192,0,88887,117722.63,21.629166,3095.281826,4099.415181
5,C10-0000786-LIC,33.060463,294,0,100044,114572.00,26.946449,3026.091885,3465.529162
6,C10-0000774-LIC,31.561922,342,0,87941,99729.00,24.713350,2786.300513,3159.788538
7,C9-0000323-LIC,30.402175,423,0,49844,61684.00,33.828391,1639.487976,2028.933799
8,MR283369,45.944674,194,0,40144,91715.08,19.341272,873.746538,1996.206996
9,402-00840,32.577504,331,0,13451,68481.99,16.598327,412.892289,2102.125165


In [60]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('license_number', 15, True)

,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,402R-00545,19.454344,648,0,199789,605898.41,17.085240,10269.634205,31144.632768
1,MR282376,41.379491,349,0,243215,516438.49,22.574598,5877.670116,12480.542234
2,C10-0000169-LIC,28.164888,509,0,268339,308263.30,29.078402,9527.429890,10944.950150
3,C10-0000408-LIC,28.716933,192,0,88887,117722.63,21.629166,3095.281826,4099.415181
4,C10-0000695-LIC,38.421989,338,0,123046,135599.21,27.606163,3202.489114,3529.208539
5,C10-0000786-LIC,33.060463,294,0,100044,114572.00,26.946449,3026.091885,3465.529162
6,C10-0000774-LIC,31.561922,342,0,87941,99729.00,24.713350,2786.300513,3159.788538
7,402-00840,32.577504,331,0,13451,68481.99,16.598327,412.892289,2102.125165
8,C9-0000323-LIC,30.402175,423,0,49844,61684.00,33.828391,1639.487976,2028.933799
9,MR283369,45.944674,194,0,40144,91715.08,19.341272,873.746538,1996.206996


In [61]:
analyzer.find_most_valuable_products_by_velocity_sales_weighted('license_number', 15)

,license_number,avg_days_since_sale,max_days_since_sale,min_days_since_sale,number_of_sales,quantity_sold,avg_sale_price,sale_velocity_per_day,quantity_sale_velocity_per_day
0,402R-00545,19.454344,648,0,199789,605898.41,17.085240,10269.634205,31144.632768
1,C10-0000169-LIC,28.164888,509,0,268339,308263.30,29.078402,9527.429890,10944.950150
2,MR282376,41.379491,349,0,243215,516438.49,22.574598,5877.670116,12480.542234
3,C10-0000695-LIC,38.421989,338,0,123046,135599.21,27.606163,3202.489114,3529.208539
4,C10-0000408-LIC,28.716933,192,0,88887,117722.63,21.629166,3095.281826,4099.415181
5,C10-0000786-LIC,33.060463,294,0,100044,114572.00,26.946449,3026.091885,3465.529162
6,C10-0000774-LIC,31.561922,342,0,87941,99729.00,24.713350,2786.300513,3159.788538
7,C9-0000323-LIC,30.402175,423,0,49844,61684.00,33.828391,1639.487976,2028.933799
8,MR283369,45.944674,194,0,40144,91715.08,19.341272,873.746538,1996.206996
9,402-00840,32.577504,331,0,13451,68481.99,16.598327,412.892289,2102.125165


In [62]:
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,10099604,0.00,0.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.758904,0.00,1,Flower,0.5,True
1,10101288,0.00,20.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.917808,0.20,1,Flower,0.5,True
2,10101795,0.00,10.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,1.068493,0.10,1,Flower,0.5,True
3,10525023,25.63,1.0,Fuzzies - Naked Mini Live Resin - Packaged - K...,Pre-Roll Infused,0.276712,25.63,1,Flower,0.5,False
4,10570444,0.00,2.0,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),1.260274,18.14,1,Edibles,1.0,True


In [63]:
valuation_1

2303332.368120699

In [64]:
df_v2, valuation_2 = analyzer.find_inventory_valuation('discount_valuation', by_product_name=True, discount_rate=.2)
df_v2.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.000000,10099604,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.0,0.758904,2.799383,Flower,0.5,True
1,0.000000,10101288,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,20.0,0.917808,2.799383,Flower,0.5,True
2,0.000000,10101795,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,10.0,1.068493,2.799383,Flower,0.5,True
3,33.255521,10525023,Fuzzies - Naked Mini Live Resin - Packaged - K...,Pre-Roll Infused,1.0,0.276712,34.976325,Flower,0.5,False
4,0.000000,10570444,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),2.0,1.260274,9.953521,Edibles,1.0,True


In [65]:
valuation_2

2384380.1637382163

In [66]:
analyzer.update_class_attributes(COMPANY_IDENTIFIER_LIST, license_numbers, True, '2022-01-01', '2022-06-01')

In [67]:
df_v1, valuation_1 = analyzer.find_inventory_valuation('last_sale_valuation', by_product_name=True, discount_rate=.2)
df_v1.head()

,package_id,price_per_unit,quantity,product_name,product_category_name,inventory_year_diff,last_sold_price_valuation,day_sold_rank,product_type,shelf_life,expired
0,10099604,0.0,0.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.758904,0.00,1,Flower,0.5,True
1,10101288,0.0,20.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,0.917808,0.20,1,Flower,0.5,True
2,10101795,0.0,10.0,[Fun Uncle] Preroll - Don Zkittlez,Pre-Roll Flower,1.068493,0.10,1,Flower,0.5,True
3,10525023,23.5,1.0,Fuzzies - Naked Mini Live Resin - Packaged - K...,Pre-Roll Infused,0.276712,23.50,1,Flower,0.5,False
4,10570444,0.0,2.0,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),1.260274,17.64,1,Edibles,1.0,True


In [68]:
valuation_1

6070727.225566418

In [69]:
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,11.885011,10525023,Fuzzies - Naked Mini Live Resin - Packaged - K...,Pre-Roll Infused,1.0,0.276712,12.5,Flower,0.5,False
1,0.000000,10570444,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),2.0,1.260274,5.0,Edibles,1.0,True
2,0.000000,10570607,[Deli] Nickels - Mango Gummy Rounds,Edible (weight - each),4.0,1.167123,5.0,Edibles,1.0,True
3,0.000000,10773302,[Deli] Nickels - Sour Green Apple Gummy Rounds,Edible (weight - each),1.0,1.241096,5.0,Edibles,1.0,True
4,0.000000,10773303,[Deli] Nickels - Sour Green Apple Gummy Rounds,Edible (weight - each),1.0,1.147945,5.0,Edibles,1.0,True


In [70]:
valuation

2865778.5049644066

In [71]:
df_i1, valuation = analyzer.find_incoming_inventory_valuation('discount_valuation', False, discount_rate=.2)
df_i1.head()

,discounted_price,package_id,product_name,product_category_name,quantity,inventory_year_diff,price_per_unit,product_type,shelf_life,expired
0,0.008425,11206434,Banana OG - Budder 1g : Sticky Farms : Bear Labs,Extract (weight - each),10.0,0.939726,0.01,Concentrates,1.0,False
1,0.000000,17267757,[Fun Uncle] OG Dipstick (Eighth | 3.5g),Flower (packaged eighth - each),8.0,0.687671,14.00,Flower,0.5,True
2,0.000000,17267758,[Fun Uncle] OG Dipstick (Eighth | 3.5g),Flower (packaged eighth - each),13.0,0.731507,14.00,Flower,0.5,True
3,NaN,17680777,JFG119-2Flid,Shake (Packaged Half Ounce - each),14.0,0.463014,NaN,Flower,0.5,False
4,0.000000,17680780,BAC160-B8e,Flower (packaged eighth - each),2.0,0.520548,NaN,Flower,0.5,True


In [72]:
valuation

2673280.512987011

In [73]:
analyzer.find_all_valuation(True, discount_rate=.2)


#### Inventory Valuation based on sales data using last sale method: $6070727.0 ####
#### Inventory Valuation based on sales data using discount time method: $5991648.0 ####
#### Inventory Valuation based on incoming data using discount time method: $2865779.0 ####


(6070727.0, 5991648.0, 2865779.0)

In [74]:
analyzer.find_all_valuation(False, discount_rate=.2)


#### Inventory Valuation based on sales data using last sale method: $3955315.0 ####
#### Inventory Valuation based on sales data using discount time method: $3842895.0 ####
#### Inventory Valuation based on incoming data using discount time method: $2673281.0 ####


(3955315.0, 3842895.0, 2673281.0)

In [75]:
# Time filter only on sales not incoming
# Weighted days sold since incoming and weighted total sales
# Research on one company 
